# Heroes of Pymoli Data Analysis 
## Melvin Garcia

In [1]:
# Import Dependencies

import pandas as pd
import os

In [2]:
# load data with purchase data 1
file = os.path.join('..', 'HeroesofPymoli', 'purchase_data2.json')

df = pd.read_json(file)

In [3]:
# Peek at data

df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


## Player Count

In [4]:
player_count = len(df['SN'].unique())
total_players= {'Total Players': [player_count]}
total_players_df = pd.DataFrame(total_players)
total_players_df

,Total Players
0,74


## Purchasing Analysis (Total)

In [5]:
unique_item_num = len(df['Item Name'].unique())
avg_purchase_price = round(df['Price'].mean(), 2)
total_num_purchase = len(df['Price'])
total_revenue = round(df['Price'].sum(), 2)

purchasing_dict = {'Number of Unique Items': [unique_item_num],
                'Average Price': ['$'+str(avg_purchase_price)],
                'Number of Purchases': [total_num_purchase],
                'Total Revenue': ['$'+str(total_revenue)]}

purchasing_df = pd.DataFrame(purchasing_dict)

purchasing_df

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.92,78,63,$228.1


## Gender Demographics

In [6]:
df_del_dup = df.drop_duplicates(['SN'], keep='last')

male_count = len(df_del_dup.loc[df_del_dup['Gender'] == 'Male'])
female_count = len(df_del_dup.loc[df_del_dup['Gender'] == 'Female'])
other_count = len(df_del_dup.loc[df_del_dup['Gender'] == 'Other / Non-Disclosed'])

total = male_count + female_count + other_count

gender_count = {'Male': [round((male_count/total)*100, 2), male_count],
                  'Female': [round((female_count/total)*100, 2), female_count],
                  'Other / Non-Disclosed' : [round((other_count/total)*100, 2), other_count]}

cols = ['Percentage of Players', 'Total Count']

gender_count_df = pd.DataFrame.from_items(gender_count.items(), 
                                          orient='index',
                                         columns = cols)

gender_count_df

,Percentage of Players,Total Count
Male,81.08,60
Female,17.57,13
Other / Non-Disclosed,1.35,1


## Purchasing Analysis (Gender)

In [7]:
# Set values

# Set male and female purchase count
gen_purchase_count = df.groupby(['Gender']).count()['Price']

# Set Average Purchase Price

gen_avg_purchase = round(df.groupby(['Gender']).mean()['Price'], 2)

# Set Total Purchase Value

gen_total_purchase = df.groupby(['Gender']).sum()['Price']

# Set normalized totals

gen_normalized = round(gen_total_purchase / gender_count_df['Total Count'], 2)

# Create df

purchase_gen_analysis = {'Purchase Count' : gen_purchase_count,
                           'Average Purchase Count' : gen_avg_purchase,
                           'Total Purchase Value': gen_total_purchase,
                           'Normalized Totals': gen_normalized}

purchase_gen_analysis_df = pd.DataFrame(purchase_gen_analysis)


# Formatting to $

purchase_gen_analysis_df['Average Purchase Count'] = purchase_gen_analysis_df['Average Purchase Count'].map("${:.2f}".format)

purchase_gen_analysis_df['Total Purchase Value'] = purchase_gen_analysis_df['Total Purchase Value'].map("${:.2f}".format)

purchase_gen_analysis_df['Normalized Totals'] = purchase_gen_analysis_df['Normalized Totals'].map("${:.2f}".format)


# Display data

purchase_gen_analysis_df

,Average Purchase Count,Normalized Totals,Purchase Count,Total Purchase Value
Gender,,,,
Female,$3.18,$3.18,13,$41.38
Male,$2.88,$3.08,64,$184.60
Other / Non-Disclosed,$2.12,$2.12,1,$2.12


## Age Demographics

In [8]:
# Create bins to organize data

bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 1000000000]

age_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Organize data according to set bins and create df

count_per_age = pd.cut(df_del_dup['Age'], bins, labels=age_labels).value_counts().rename('Total Count')
count_age_df = pd.DataFrame(count_per_age)


percent_by_age = round(count_per_age/ count_per_age.sum()*100, 2).rename('Percentage of Players')
percent_age_df = pd.DataFrame(percent_by_age)

# Create df

age_demo_df = pd.concat([percent_age_df, count_age_df], axis=1).reindex(age_labels)

age_demo_df

,Percentage of Players,Total Count
<10,6.76,5
10-14,4.05,3
15-19,14.86,11
20-24,45.95,34
25-29,10.81,8
30-34,8.11,6
35-39,8.11,6
40+,1.35,1


## Purchasing Analysis (Age)

In [9]:
# Filter main df by age groups
df.loc[(df['Age'] < 10), 'Age_label'] = '<10'

df.loc[(df['Age'] >= 10) & (df['Age'] <= 14), 'Age_label'] = '10-14'

df.loc[(df['Age'] >= 15) & (df['Age'] <= 19), 'Age_label'] = '15-19'

df.loc[(df['Age'] >= 20) & (df['Age'] <= 24), 'Age_label'] = '20-24'

df.loc[(df['Age'] >= 25) & (df['Age'] <= 29), 'Age_label'] = '25-29'

df.loc[(df['Age'] >= 30) & (df['Age'] <= 34), 'Age_label'] = '30-34'

df.loc[(df['Age'] >= 35) & (df['Age'] <= 39), 'Age_label'] = '35-39'

df.loc[(df['Age'] >= 40), 'Age_label'] = '40+'


# Set count per age group
age_purchase_count = df.groupby(['Age_label']).count()['SN']

# Set avg purchase per age group
age_avg_purchase = df.groupby(['Age_label'])['Price'].mean()

# Set total purchase per age group
age_total_purchase = df.groupby(['Age_label'])['Price'].sum().reindex(age_labels)

# Set normalized total purchase per age group
age_normalized = age_total_purchase / age_demo_df['Total Count']

# # Create dict to create df

purchase_age_analysis = {'Purchase Count' : age_purchase_count,
                           'Average Purchase Count' : age_avg_purchase,
                           'Total Purchase Value': age_total_purchase,
                           'Normalized Totals': age_normalized}

purchase_age_analysis_df = pd.DataFrame(purchase_age_analysis).reindex(age_labels)

# Formatting to $ 

purchase_age_analysis_df['Average Purchase Count'] = purchase_age_analysis_df['Average Purchase Count'].map("${:.2f}".format)

purchase_age_analysis_df['Total Purchase Value'] = purchase_age_analysis_df['Total Purchase Value'].map("${:.2f}".format)

purchase_age_analysis_df['Normalized Totals'] = purchase_age_analysis_df['Normalized Totals'].map("${:.2f}".format)


# Display data

purchase_age_analysis_df

,Average Purchase Count,Normalized Totals,Purchase Count,Total Purchase Value
<10,$2.76,$2.76,5,$13.82
10-14,$2.99,$2.99,3,$8.96
15-19,$2.76,$2.76,11,$30.41
20-24,$3.02,$3.20,36,$108.89
25-29,$2.90,$3.26,9,$26.11
30-34,$1.98,$2.31,7,$13.89
35-39,$3.56,$3.56,6,$21.37
40+,$4.65,$4.65,1,$4.65


## Top Spenders

In [10]:
# Identify top 5 spenders

top5_spenders = df.groupby(['SN'])['Price'].sum().sort_values(ascending=False).head().index

top5_SN_df = df.loc[df['SN'].isin(top5_spenders)]

# Organize data

top5_count = top5_SN_df.groupby(['SN']).count()['Price']

top5_avg = top5_SN_df.groupby(['SN'])['Price'].mean()

top5_total = top5_SN_df.groupby(['SN'])['Price'].sum()

# Create dict to create df

purchase_top5_analysis = {'Purchase Count' : top5_count,
                           'Average Purchase Count' : top5_avg,
                           'Total Purchase Value': top5_total}

purchase_top5_analysis_df = pd.DataFrame(purchase_top5_analysis).sort_values(['Purchase Count'], ascending=False)

# Formatting with $ 
purchase_top5_analysis_df['Average Purchase Count'] = purchase_top5_analysis_df['Average Purchase Count'].map("${:.2f}".format)

purchase_top5_analysis_df['Total Purchase Value'] = purchase_top5_analysis_df['Total Purchase Value'].map("${:.2f}".format)

purchase_top5_analysis_df

,Average Purchase Count,Purchase Count,Total Purchase Value
SN,,,
Aidaira26,$2.56,2,$5.13
Sundaky74,$3.71,2,$7.41
Alarap40,$4.71,1,$4.71
Chanirra64,$4.78,1,$4.78
Eusty71,$4.81,1,$4.81


## Most Popular Items

In [11]:
# Organize data

top_items = df['Item ID'].value_counts().head().index

top_items_df = df.loc[df['Item ID'].isin(top_items)]

top_items_count = top_items_df.groupby(['Item ID', 'Item Name']).count()['Price']

top_items_price = top_items_df.groupby(['Item ID', 'Item Name']).Price.unique().astype(float)

top_items_total = top_items_df.groupby(['Item ID', 'Item Name'])['Price'].sum()

# Create dict to create df

top_items_analysis = {'Purchase Count' : top_items_count,
                           'Item Price' : top_items_price,
                           'Total Purchase Value': top_items_total}

top_items_analysis_df = pd.DataFrame(top_items_analysis).sort_values(['Purchase Count'], ascending=False)

# Formatting to $

top_items_analysis_df['Item Price'] = top_items_analysis_df['Item Price'].map("${:.2f}".format)
top_items_analysis_df['Total Purchase Value'] = top_items_analysis_df['Total Purchase Value'].map("${:.2f}".format)

# Display data

top_items_analysis_df

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
94,Mourning Blade,$3.64,3,$10.92
60,Wolf,$2.70,2,$5.40
64,Fusion Pummel,$2.42,2,$4.84
93,Apocalyptic Battlescythe,$4.49,2,$8.98
98,"Deadline, Voice Of Subtlety",$1.29,2,$2.58


## Most Profitable Items

In [12]:
# Organize data

top5_profit = pd.DataFrame(df.groupby(['Item ID'])['Price'].sum()).sort_values('Price', ascending=False)

top5_profit = top5_profit.head().index

top5_profit_df = df.loc[df['Item ID'].isin(top5_profit)]

top5_profit_count = top5_profit_df.groupby(['Item ID', 'Item Name']).count()['Price']

top5_profit_price = top5_profit_df.groupby(['Item ID', 'Item Name']).Price.unique().astype(float)

top5_profit_total = top5_profit_df.groupby(['Item ID', 'Item Name'])['Price'].sum()

# Create dict to create df

top5_profit_analysis = {'Purchase Count' : top5_profit_count,
                           'Item Price' : top5_profit_price,
                           'Total Purchase Value': top5_profit_total}

top5_profit_analysis_df = pd.DataFrame(top5_profit_analysis).sort_values(['Purchase Count'], ascending=False)

# Formatting to $

top5_profit_analysis_df['Item Price'] = top5_profit_analysis_df['Item Price'].map("${:.2f}".format)
top5_profit_analysis_df['Total Purchase Value'] = top5_profit_analysis_df['Total Purchase Value'].map("${:.2f}".format)

# Display data

top5_profit_analysis_df

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
94,Mourning Blade,$3.64,3,$10.92
90,Betrayer,$4.12,2,$8.24
93,Apocalyptic Battlescythe,$4.49,2,$8.98
117,"Heartstriker, Legacy of the Light",$4.71,2,$9.42
154,Feral Katana,$4.11,2,$8.22
